# Use Pandas for ETL

In [ ]:
import pandas as pd
import io

In [ ]:
# caricato manualmente:
# caricare "EXPERIENCE_x_import_csv"

ds_experience = pd.read_csv('./EXPERIENCE_x_import_csv.csv')
# pd.read_sql("select campo, count(*) from tabella group by campo")

FileNotFoundError: ignored

In [ ]:
ds_experience.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72667 entries, 0 to 72666
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   SOCIAL_PROFILE_ID    72667 non-null  int64  
 1   PREDICTED_GENDER_ID  70852 non-null  float64
 2   RAW_TITLE            72667 non-null  object 
 3   RAW_EXPERIENCE       72434 non-null  object 
 4   RAW_COUNTRY          55032 non-null  object 
 5   RAW_INDUSTRY         47837 non-null  object 
 6   RAW_CITY             51447 non-null  object 
 7   RAW_REGION           54923 non-null  object 
 8   RAW_START_DATE       72103 non-null  object 
 9   RAW_END_DATE         54886 non-null  object 
 10  RAW_EMPLOYER         72416 non-null  object 
 11  IS_CURRENT           72448 non-null  object 
 12  IS_MOST_RECENT       72448 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 7.2+ MB


In [ ]:
ds_experience.head()

SOCIAL_PROFILE_ID  PREDICTED_GENDER_ID  \
0           35459827                  NaN   
1           67202826                  1.0   
2           57663781                  0.0   
3           60134055                  0.0   
4           11389420                  1.0   

                                           RAW_TITLE  \
0                                       Data Analyst   
1      Data Science and Machine Learning at CARDO AI   
2                     Project Manager & Data Analyst   
3  R&D Project Analyst, Data Science Strategy Con...   
4  🎯 10+ years solving problems at the nexus of d...   

                       RAW_EXPERIENCE RAW_COUNTRY     RAW_INDUSTRY  \
0     Direzione Commerciale Insurance       Italy              NaN   
1     The Data Incubator/Reply Fellow       Italy              NaN   
2  Training Consultant at Barilla LAB         NaN              NaN   
3   Downlink” Operator For MRO/SHARAD       Italy  Defense & Space   
4                  Visiting Professor       Spain     Online Media   

     RAW_CITY RAW_REGION RAW_START_DATE   RAW_END_DATE  \
0       Turin       EMEA   January 2019       May 2019   
1  Milan Area       EMEA      June 2017      July 2017   
2         NaN        NaN   January 2007  December 2008   
3        Rome       EMEA   January 2015     April 2016   
4      Madrid       EMEA           2015            NaN   

                                   RAW_EMPLOYER IS_CURRENT IS_MOST_RECENT  
0                     Omicron Consulting S.r.L.      False          False  
1                                         Reply      False          False  
2                                 Barilla Group      False          False  
3  Università Degli Studi Di Roma La Sapienza ​      False          False  
4                                      El Mundo      False          False

In [ ]:
number_of_records = ds_experience.shape[0]
print(f"Number of records loaded {number_of_records}")

Number of records loaded 72667


In [ ]:
# PRIMA DI PROCEDERE ANALIZZIAMO UN PO' I DATI

In [ ]:
# conta valori unici della colonna ID (ogni valore una volta, regardless di quante volte compare).
# formula: df.nome_colonna.nunique()
ds_experience.SOCIAL_PROFILE_ID.nunique()

# quindi sono 15484 profili. coincide con conta del dataset EDUCATION di Claudia


15484

In [ ]:
# GENDER
# come contare valori 1 (maschi) della colonna PREDICTED_GENDER_ID raggruppando per valori unici colonna ID

gender_m = ds_experience[ds_experience['PREDICTED_GENDER_ID'] == 1].groupby('SOCIAL_PROFILE_ID')['PREDICTED_GENDER_ID'].count()

print(gender_m)

# trasformiamo la colonna unique_counts in un DataFrame,x poi contare quanti record ci sono
df_gender_m = pd.DataFrame(gender_m)
print(df_gender_m.head())
print(df_gender_m.shape[0])

SOCIAL_PROFILE_ID
13389        5
91526        1
166049       3
221793       4
243809       7
            ..
623558635    4
623584860    4
623587796    1
623625082    2
623714658    7
Name: PREDICTED_GENDER_ID, Length: 9501, dtype: int64
                   PREDICTED_GENDER_ID
SOCIAL_PROFILE_ID                     
13389                                5
91526                                1
166049                               3
221793                               4
243809                               7
9501


In [ ]:
# GENDER
# come contare valori 0 (femmine) della colonna PREDICTED_GENDER_ID raggruppando per valori unici colonna ID

gender_f = ds_experience[ds_experience['PREDICTED_GENDER_ID'] == 0].groupby('SOCIAL_PROFILE_ID')['PREDICTED_GENDER_ID'].count()

print(gender_f)
print("******************************")
# trasformiamo la tabellina in un DataFrame,x poi contare quanti record ci sono
df_gender_f = pd.DataFrame(gender_f)
print(df_gender_f.head())
print("******************************")
print(df_gender_f.shape[0])

SOCIAL_PROFILE_ID
42765        2
90190        4
114502       3
430650       4
622875       1
            ..
623404648    6
623417971    1
623443830    1
623496406    2
623732754    1
Name: PREDICTED_GENDER_ID, Length: 5565, dtype: int64
******************************
                   PREDICTED_GENDER_ID
SOCIAL_PROFILE_ID                     
42765                                2
90190                                4
114502                               3
430650                               4
622875                               1
******************************
5565


In [ ]:
# GENDER null:
gen_null = ds_experience.groupby('SOCIAL_PROFILE_ID')['PREDICTED_GENDER_ID'].apply(lambda x: x.isnull().sum())
print(gen_null)

SOCIAL_PROFILE_ID
13389        0
42765        0
90190        0
91526        0
114502       0
            ..
623584860    0
623587796    0
623625082    0
623714658    0
623732754    0
Name: PREDICTED_GENDER_ID, Length: 15484, dtype: int64


In [ ]:
# da chatGPT how to count rows with a value different from another value pandas

df_gen_null = pd.DataFrame(gen_null)
print(df_gen_null.head())
# null_count:
filtered_df = df_gen_null[df_gen_null['PREDICTED_GENDER_ID'] != 0]
print(len(filtered_df))


                   PREDICTED_GENDER_ID
SOCIAL_PROFILE_ID                     
13389                                0
42765                                0
90190                                0
91526                                0
114502                               0
401


In [ ]:
# quale è la max e min occorrenza dei profili? il prfilo 109640761 compare 55 volte!
print(ds_experience['SOCIAL_PROFILE_ID'].value_counts().max())
print(ds_experience['SOCIAL_PROFILE_ID'].value_counts().min())
print(ds_experience['SOCIAL_PROFILE_ID'].value_counts().idxmax())  #most common value in column

55
1
109640761


In [ ]:
# contiamo media dell'occorrenza dei profili, cioè in media quante esperienze ha un profilo:

valori = ds_experience['SOCIAL_PROFILE_ID'].value_counts()
print(valori)

109640761    55
349081215    50
198716267    35
304084617    33
71703563     31
             ..
506995126     1
538491000     1
592667266     1
622195613     1
185289699     1
Name: SOCIAL_PROFILE_ID, Length: 15484, dtype: int64


In [ ]:
# trasformiamo la tabellina valori in un DataFrame,x poi contare la media
df_valori = pd.DataFrame(valori)
# df_value_counts.rename(columns={None: "value_count"}, inplace=True)
print(df_valori.head())


           SOCIAL_PROFILE_ID
109640761                 55
349081215                 50
198716267                 35
304084617                 33
71703563                  31


In [ ]:
# X METTERE ORDINE:
# Define a dictionary to rename the columns
new_column_names = {
    "SOCIAL_PROFILE_ID": "ID",
    None: "occurr"
}

# Rename all columns using the rename() method
df_valori.rename(columns=new_column_names, inplace=True)

print(df_valori.head(150))

           ID
109640761  55
349081215  50
198716267  35
304084617  33
71703563   31
...        ..
583191682  16
128482734  16
291513682  16
382682744  16
324576512  16

[150 rows x 1 columns]


In [ ]:
print(df_valori['ID'].mean())

4.693037974683544


In [ ]:
# come contare valori unici della colonna raw_title, raggruppati per (group by)
# valori unici colonna ID, per vedere se ogni ID ha solo un raw_title.

unique_counts = ds_experience.groupby('SOCIAL_PROFILE_ID')['RAW_TITLE'].nunique()

print(unique_counts)
print(unique_counts.dtype)

SOCIAL_PROFILE_ID
13389        1
42765        1
90190        1
91526        1
114502       1
            ..
623584860    1
623587796    1
623625082    1
623714658    1
623732754    1
Name: RAW_TITLE, Length: 15484, dtype: int64
int64


In [ ]:
# trasformiamo la colonna unique_counts in un DataFrame,x poi contare quanti valori
# diversi compaiono nella colonna RAW_TITLE e quindi capire se ci sono profili con piu di un raw_title
df_unique_counts = pd.DataFrame(unique_counts)
print(df_unique_counts.head())
df_unique_counts.RAW_TITLE.nunique()
print('*******************')
null_count = df_unique_counts['RAW_TITLE'].isnull().sum()
print(null_count)

                   RAW_TITLE
SOCIAL_PROFILE_ID           
13389                      1
42765                      1
90190                      1
91526                      1
114502                     1
*******************
0


In [ ]:
# RAW_TITLE presenta solo un valore, ed è sicuramente "1", perche già lo vediamo nell' head()
# quindi tutti i profili hanno un solo titolo, che sarà il titolo attuale dichiarato.
# x country, ordinando gli ID in google sheets, si capisce che si riferisce alla country dell'esperienza

In [ ]:
# come contare occurrences del valore TRUE nella colonna IS_MOST_RECENT, raggruppando per ID. facciamo cio x vedere
# quanti hanno dichiarato ultima esperienza, in modo da vedere da dove provengono

occurrences_count = ds_experience[ds_experience['IS_MOST_RECENT'] == True].groupby('SOCIAL_PROFILE_ID')['IS_MOST_RECENT'].count()

print(occurrences_count)

SOCIAL_PROFILE_ID
13389        1
42765        1
90190        1
91526        1
114502       1
            ..
623584860    2
623587796    1
623625082    1
623714658    1
623732754    1
Name: IS_MOST_RECENT, Length: 15265, dtype: int64


In [ ]:
# come prima: trasformiamo in un DF la colonna occurrencies_counts x poi estrapolare i valori diversi da 1
# e quindi capire se ci sono profili con piu (o meno) di una esperienza dichiarata come piu recente
df_occurrences_count = pd.DataFrame(occurrences_count)
print(df_occurrences_count.head())
df_occurrences_count.IS_MOST_RECENT.nunique()

                   IS_MOST_RECENT
SOCIAL_PROFILE_ID                
13389                           1
42765                           1
90190                           1
91526                           1
114502                          1


8

In [ ]:
null_count = df_occurrences_count['IS_MOST_RECENT'].isnull().sum()
print(null_count)

0


In [ ]:
# 8 hanno dichiarato come MOST_RECENT un numero di esperienze diverso da 1. essendo solo 8 su 15k+, si possono ignorare.
# per completezza calcoliamo anche che ci sono 0 null.

In [ ]:
# vediamo ora chi non dichiara NESSUNA ESPERIENZA come IS_CURRENT, e quindi in teoria è UNEMPLOYED.
# per farlo ragioniamo al contrario: conto quanti IS_CURRENT sono True, sempre groupby(ID), e relaziono il risultato
# al n degli ID (contati 1 volta).
# piu IS_CURRENT sono "True" in proporzione al numero degli ID(contati una sola volta), minore è la %
# di utenti che si dichiarano unemployed.

unemployed_count = ds_experience[ds_experience['IS_CURRENT'] == True].groupby('SOCIAL_PROFILE_ID')['IS_CURRENT'].count()
# in altre parole: per ogni ID, quanti IS_CURRENT sono True

print(unemployed_count)

SOCIAL_PROFILE_ID
13389        1
42765        1
91526        1
114502       1
221793       1
            ..
623553924    1
623558635    1
623584860    2
623714658    1
623732754    1
Name: IS_CURRENT, Length: 13761, dtype: int64


In [ ]:
# come prima,  ma questa volta contiamo le occorrenze del valore 0, e cioè chi ha dichiarato 0 True in "IS_CURRENT"
df_unemployed_count = pd.DataFrame(unemployed_count)
print(df_unemployed_count.head())


                   IS_CURRENT
SOCIAL_PROFILE_ID            
13389                       1
42765                       1
91526                       1
114502                      1
221793                      1


In [ ]:
# formula da chatgpt: occurrences_count = df['Column_A'].value_counts()[value_to_count]
disoccupati_count = df_unemployed_count['IS_CURRENT'].value_counts()[0]

print(disoccupati_count)

# cercando il valore 0 dà errore: ma cercando i valori 1, 2, 3 etc.... restituisce il n di persone che hanno dichiarato
# 1, 2, 3 etc esperienze come "IS_CURRENT".

KeyError: 0

In [ ]:
# formula da chatgpt: occurrences_count = df['Column_A'].value_counts()[value_to_count]
disoccupati_count = df_unemployed_count['IS_CURRENT'].value_counts()[1]

print(disoccupati_count)


13442


In [ ]:
# formula da chatgpt: occurrences_count = df['Column_A'].value_counts()[value_to_count]
disoccupati_count = df_unemployed_count['IS_CURRENT'].value_counts()[2]

print(disoccupati_count)


276


In [ ]:
null_count = df_unemployed_count['IS_CURRENT'].isnull().sum()
print(null_count)

0


In [ ]:
# in alternativa, possiamo comtare quanti valori diversi da zero compaiono nella colonna del nuovo
# dataframe "df_unemployed_count", e anche quelli uguali a zero:

different_value_count = (df_unemployed_count['IS_CURRENT'] != 0).sum()
print(different_value_count)

zero_value_count = (df_unemployed_count['IS_CURRENT'] == 0).sum()
print(zero_value_count)

13761
0


In [ ]:
# 0 si dichiarano disoccupati, 13761 dichiarano almeno una "IS_CURRENT". manca qualcuno all'appello.

In [ ]:
# nemmeno con la conta dei null ho risolto...
null_count = df_unemployed_count['IS_CURRENT'].isnull().sum()
print(null_count)

notnull_count = df_unemployed_count['IS_CURRENT'].notnull().sum()
print(notnull_count)

0
13761


In [ ]:
# guardando da excel/sheets  ho visto che alcuni ID hanno tutte o quasi tutte le info mancanti/null:
# ad esempio l'ID 226976994, che corrisponde (NEL DATAFRAME NON ORDINATO, QUELLO ORIGINALE) al record n. 18218:

filtered_df = ds_experience[ds_experience['SOCIAL_PROFILE_ID'] == 226976994]
print(filtered_df['IS_CURRENT'])


18218    NaN
Name: IS_CURRENT, dtype: object


In [ ]:
# ha tutto missing...
print(ds_experience.iloc[18218])

SOCIAL_PROFILE_ID                                              226976994
PREDICTED_GENDER_ID                                                  1.0
RAW_TITLE              Laurea triennale in Informatica. Attuale stude...
RAW_EXPERIENCE                                                       NaN
RAW_COUNTRY                                                          NaN
RAW_INDUSTRY                                                         NaN
RAW_CITY                                                             NaN
RAW_REGION                                                           NaN
RAW_START_DATE                                                       NaN
RAW_END_DATE                                                         NaN
RAW_EMPLOYER                                                         NaN
IS_CURRENT                                                           NaN
IS_MOST_RECENT                                                       NaN
Name: 18218, dtype: object


In [ ]:
# cambiamo le colonne START e END in date con formato mm-aaaa:
# df['DateColumn'] = pd.to_datetime(df['DateColumn'], format='%B %Y')
# df['DateColumn'] = df['DateColumn'].dt.strftime('%m-%Y')

ds_experience['RAW_START_DATE'] = pd.to_datetime(ds_experience['RAW_START_DATE'], format='%B %Y')
ds_experience['RAW_START_DATE'] = ds_experience['RAW_START_DATE'].dt.strftime('%m-%Y')


ValueError: time data '2015' does not match format '%B %Y' (match)

In [ ]:
ds_experience['RAW_END_DATE'] = pd.to_datetime(ds_experience['RAW_END_DATE'], format='%B %Y')
ds_experience['RAW_END_DATE'] = ds_experience['RAW_END_DATE'].dt.strftime('%m-%Y')

ValueError: time data '2014' does not match format '%B %Y' (match)

In [ ]:
# ***TOKENIZZAZIONE***

In [ ]:
pip install pandas nltk

In [ ]:
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords

from nltk.stem import PorterStemmer, WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zeta9\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words = set(stopwords.words('italian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zeta9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Carica il file CSV in un DataFrame pandas
# data = pd.read_csv('EXPERIENCE_x_import.csv') lo avevamo gia fatto all'inizio del notebook, e il df si chiama:
# ds_experience


# Uniamo tutte le righe della colonna 'RAW_TITLE' in un unico testo
all_text = ' '.join(ds_experience['RAW_TITLE'].astype(str).tolist())

In [ ]:
# Tokenizzazione del testo
tokens = word_tokenize(all_text)

filtered_tokens = [word.lower() for word in tokens if word.lower() not in stop_words and word.isalpha()]


In [ ]:
# Calcolo della distribuzione di frequenza delle parole
fdist = FreqDist(filtered_tokens)

# Estrai le parole più frequenti (ad esempio, le prime 100 parole)
most_common_words = fdist.most_common(100)

print("Parole più frequenti:")
for word, frequency in most_common_words:
    print(f"{word}: {frequency}")

In [ ]:
# ***LEMMATIZZAZIONE***

In [ ]:
# Assicurati di scaricare le stopwords e il WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
# Uniamo tutte le righe della colonna 'RAW-TITLE' in un unico testo
all_text = ' '.join(ds_experience['RAW_TITLE'].astype(str).tolist())
# Tokenizzazione del testo
tokens = word_tokenize(all_text)

In [ ]:
stop_words_italian = set(stopwords.words('italian'))
stop_words_english = set(stopwords.words('english'))
filtered_tokens = [word.lower() for word in tokens if word.lower() not in stop_words_italian and word.lower() not in stop_words_english and word.isalpha()]

In [ ]:
nltk.download('omw-1.4')

# Stemming
stemmer = PorterStemmer()
stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
# Lemmatizzazione
lemmatizer = WordNetLemmatizer()
lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\zeta9\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
# Calcolo della distribuzione di frequenza delle parole (dopo stemming)
fdist_stemmed = FreqDist(stemmed_tokens)
most_common_words_stemmed = fdist_stemmed.most_common(10)

print("Parole più frequenti dopo stemming:")
for word, frequency in most_common_words_stemmed:
    print(f"{word}: {frequency}")

# Calcolo della distribuzione di frequenza delle parole (dopo lemmatizzazione)
fdist_lemmatized = FreqDist(lemmatized_tokens)
most_common_words_lemmatized = fdist_lemmatized.most_common(100)

print("\nParole più frequenti dopo lemmatizzazione:")
for word, frequency in most_common_words_lemmatized:
    print(f"{word}: {frequency}")

In [ ]:
# ***KEYWORDS RAW_TITLE***

In [ ]:
# Rimuovi le righe con valori mancanti nella colonna 'RAW_TITLE'
ds_experience.dropna(subset=['RAW_TITLE'], inplace=True)

# Verifica il tipo della colonna 'RAW_TITLE'
print(ds_experience['RAW_TITLE'].dtype)  # Assicurati che sia una stringa

# Funzione per lemmatizzare e rimuovere le stopwords
def process_text(text):
    tokens = word_tokenize(text)
    stop_words_italian = set(stopwords.words('italian'))
    stop_words_english = set(stopwords.words('english'))
    filtered_tokens = [word.lower() for word in tokens if word.lower() not in stop_words_italian and word.lower() not in stop_words_english and word.isalpha()]

    # Lemmatizzazione
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

    return lemmatized_tokens

# Crea una nuova colonna nel DataFrame per le parole chiave
ds_experience['KEYWORDS'] = ds_experience['RAW_TITLE'].apply(process_text)

# Stampa il DataFrame con le parole chiave associate
print(ds_experience[['RAW_TITLE', 'KEYWORDS']])

object
                                               RAW_TITLE  \
0                                           Data Analyst   
1          Data Science and Machine Learning at CARDO AI   
2                         Project Manager & Data Analyst   
3      R&D Project Analyst, Data Science Strategy Con...   
4      🎯 10+ years solving problems at the nexus of d...   
...                                                  ...   
72662  Analytics Technical Lead | Data Science at Gen...   
72663                               Junior Data Engineer   
72664  BigData Engineer - Cloud Data Engineer presso ...   
72665  ✔️ EPC & 5GC Integration Engineer | Data Scien...   
72666  Bus & Technology Delivery Analyst at Accenture...   

                                                KEYWORDS  
0                                        [data, analyst]  
1              [data, science, machine, learning, cardo]  
2                      [project, manager, data, analyst]  
3      [r, project, analyst, data, s

In [ ]:
print(ds_experience)

       SOCIAL_PROFILE_ID  PREDICTED_GENDER_ID  \
0               35459827                  NaN   
1               67202826                  1.0   
2               57663781                  0.0   
3               60134055                  0.0   
4               11389420                  1.0   
...                  ...                  ...   
72662          361084303                  1.0   
72663          333378108                  1.0   
72664          333371236                  1.0   
72665          453204780                  1.0   
72666          499035213                  0.0   

                                               RAW_TITLE  \
0                                           Data Analyst   
1          Data Science and Machine Learning at CARDO AI   
2                         Project Manager & Data Analyst   
3      R&D Project Analyst, Data Science Strategy Con...   
4      🎯 10+ years solving problems at the nexus of d...   
...                                                

In [ ]:
print(ds_experience[['SOCIAL_PROFILE_ID', 'RAW_TITLE', 'KEYWORDS']])

       SOCIAL_PROFILE_ID                                          RAW_TITLE  \
0               35459827                                       Data Analyst   
1               67202826      Data Science and Machine Learning at CARDO AI   
2               57663781                     Project Manager & Data Analyst   
3               60134055  R&D Project Analyst, Data Science Strategy Con...   
4               11389420  🎯 10+ years solving problems at the nexus of d...   
...                  ...                                                ...   
72662          361084303  Analytics Technical Lead | Data Science at Gen...   
72663          333378108                               Junior Data Engineer   
72664          333371236  BigData Engineer - Cloud Data Engineer presso ...   
72665          453204780  ✔️ EPC & 5GC Integration Engineer | Data Scien...   
72666          499035213  Bus & Technology Delivery Analyst at Accenture...   

                                                KEY

In [ ]:
ds_experience.to_csv('processed_ds_experience.csv', index=False)
from IPython.display import FileLink
FileLink('processed_ds_experience.csv')

C:\Users\zeta9\Downloads\processed_ds_experience.csv

In [ ]:
#definisco funzione x popolare colonna STANDARD_TITLE

def create_standard_title(words_list):
    if 'analyst' in words_list or 'analist' in words_list or 'analst' in words_list or 'anlist' in words_list or 'anlyst' in words_list or 'analytics' in words_list or 'analitics' in words_list  or 'analytic' in words_list or 'analitic' in words_list or 'analysis' in words_list or 'analisis' in words_list:
        return 'data analyst'
    elif 'engineer' in words_list or 'enginer' in words_list or'engeneer' in words_list or'engener' in words_list or 'engineering' in words_list or'enginering' in words_list or'engenering' in words_list or'engeneering' in words_list:
        return 'data engineer'
    elif 'architect' in words_list or 'architet' in words_list or 'archytet' in words_list or 'archtect' in words_list or 'archytect' in words_list or 'architecture' in words_list or 'architectur' in words_list:
        return 'data architect'
    elif 'scientist' in words_list or 'scientyst' in words_list or'scyentist' in words_list or 'scientst' in words_list or 'scyentst' in words_list or 'science' in words_list or 'scienc' in words_list:
        return 'data scientist'
    else:
        return 'other'

# applico funzione:

ds_experience['STANDARD_TITLE'] = ds_experience['KEYWORDS'].apply(create_standard_title)


In [ ]:
print(ds_experience[['SOCIAL_PROFILE_ID', 'RAW_TITLE', 'KEYWORDS', 'STANDARD_TITLE']])

       SOCIAL_PROFILE_ID                                          RAW_TITLE  \
0               35459827                                       Data Analyst   
1               67202826      Data Science and Machine Learning at CARDO AI   
2               57663781                     Project Manager & Data Analyst   
3               60134055  R&D Project Analyst, Data Science Strategy Con...   
4               11389420  🎯 10+ years solving problems at the nexus of d...   
...                  ...                                                ...   
72662          361084303  Analytics Technical Lead | Data Science at Gen...   
72663          333378108                               Junior Data Engineer   
72664          333371236  BigData Engineer - Cloud Data Engineer presso ...   
72665          453204780  ✔️ EPC & 5GC Integration Engineer | Data Scien...   
72666          499035213  Bus & Technology Delivery Analyst at Accenture...   

                                                KEY

In [ ]:
ds_experience.to_csv('processed_ds_experience.csv', index=False)
from IPython.display import FileLink
FileLink('processed_ds_experience.csv')

C:\Users\zeta9\Downloads\processed_ds_experience.csv

In [ ]:
# CREIAMO TABELLA SOLO X STANDARD_TITLE. ATTRIBUTI: title_code, standard_title, associated_keywords

df_original = ds_experience

# Select specific columns to create a new DataFrame (new table)
selected_columns = ['STANDARD_TITLE', 'KEYWORDS']
df_standard_titles = pd.DataFrame(df_original[selected_columns])

# aggiungiamo colonna codici ('title_code'), poi cambiamo nome colonne:
def add_title_code(posizioni):
    if 'data analyst' in posizioni:
        return 'DAN'
    elif 'data engineer' in posizioni:
        return 'DEN'
    elif 'data scientist' in posizioni:
        return 'DSC'
    elif 'data architect' in posizioni:
        return 'DAR'
    else:
        return 'oth'

# DAN = D ANALYST, DEN = data engineer, DAR = data architect , DSC data scientist
# applico funzione:

df_standard_titles['title_code'] = df_standard_titles['STANDARD_TITLE'].apply(add_title_code)
print(df_standard_titles.df_standard_titles['title_code', 'STANDARD_TITLE', 'KEYWORDS'])

In [ ]:
print(df_standard_titles)

       STANDARD_TITLE                                           KEYWORDS  \
0        data analyst                                    [data, analyst]   
1      data scientist          [data, science, machine, learning, cardo]   
2        data analyst                  [project, manager, data, analyst]   
3        data analyst  [r, project, analyst, data, science, strategy,...   
4      data scientist  [year, solving, problem, nexus, data, science,...   
...               ...                                                ...   
72662    data analyst  [analytics, technical, lead, data, science, ge...   
72663   data engineer                           [junior, data, engineer]   
72664   data engineer  [bigdata, engineer, cloud, data, engineer, pre...   
72665   data engineer  [epc, integration, engineer, data, science, pr...   
72666    data analyst  [bus, technology, delivery, analyst, accenture...   

      title_code  
0            DAN  
1            DSC  
2            DAN  
3          